# Module: Functional Annotation

Functional annotation of sequences involves identifying the biological roles of genes or proteins by assigning them known functions based on sequence similarity, conserved motifs, and/or domain architecture. This process is a crucial step in bioinformatics pipelines, enabling researchers to interpret sequence data in the context of cellular and molecular biology. This, in turn, provides insights into the potential roles of genes within an organism or a microbial community. Additionally, computational tools and databases play a key role in annotating sequences efficiently and in a case-specific manner.

In this module, we will look into several bioinformatics tools and databases that you can use in functionally annotating your sequence data.

Created by: _Microbial Oceanography Laboratory (MOLab)_

---
## How to Use This Notebook

1. Make sure tools are installed already (see below if not yet).
2. Activate environment. Replace environment name accordingly.
```bash
conda activate func-annot-env
```
2. Open jupyter notebook with the command below and select the notebook.
```bash
jupyter notebook
```
3. To run the cells in this notebook, press Shift+Enter.

---
## Tools Used
1. **Prodigal** for prokaryotic gene prediction.
2. **Augustus** for eukaryotic gene prediction.
3. **Prokka** for prokaryotic gene prediction and annotation.
4. **BLAST** for finding gene homologs.
5. **DIAMOND** for finding gene homologs (way faster than BLAST).
6. **eggNOG Mapper** annotates genes against the eggNOG database.
7. **BlastKOALA** assigns Kegg Orthology (KO) to genes
8. **MetaEuk** for annotation of eukaryotic fraction of a metagenomic dataset.

To install tools (1), (3), (4), (5), and (8), find the `func-annot.yaml` file located in the same folder as this notebook (in repository). Then run the command below in the terminal:
```bash
conda env create -f func-annot.yaml
```

---

To install **Augustus** follow the steps below:

Create a copy of repository.
```bash
git clone https://github.com/Gaius-Augustus/Augustus.git
```

Run setup.
```bash
make augustus
```

Create a soft link.
```bash
sudo ln -s ~/path/to/Augustus/bin/augustus /usr/local/bin/augustus
```

---

Tools (6) and (7) can be accessed through webservers or EU Galaxy.

<div class="alert alert-block alert-info">
<b>Note:</b> 
    
Some tools and databases require a lot of computational resources. If you have trouble executing some of these locally, you can use an HPC or try to check if the tool (and/or database) of interest is available in the <a href='https://usegalaxy.eu/'>Galaxy EU webserver</a>.
</div>

---
## Starting Files

1. A contig FASTA file (`assembly_contigs.fasta`) generated from assembly (see **Assembly Module**) or binning (see **Binning Module**).

---
## Expected Outputs

Output(s) may depend on the tool used, but generally, the following outputs will be present:

1. FASTA file of DNA and/or amino acid sequence of predicted genes.
2. Some annotation file showing gene ID and corresponding annotation (e.g. GFF, GTF, tabular, etc).

---
## Table of Contents
 * [**Prodigal**](#Prodigal)  
 * [**AUGUSTUS**](#AUGUSTUS)  
 * [**Prokka**](#Prokka)  
 * [**BLAST**](#BLAST)
 * [**DIAMOND**](#DIAMOND)
 * [**EggNOG Mapper**](#EggNOG-Mapper)
 * [**BlastKOALA**](#BlastKOALA)
 * [**MetaEuk**](#MetaEuk)

---
# <font color = 'gray'>Prodigal</font>

`prodigal` is tool designed to predict protein-coding genes in bacterial and archaeal genomes (it also has an option optimized for shotgun metagenomes using `-p meta` option). Its input is a FASTA file of contig assembly and its primary outputs are FASTA files of the predicted genes (both translated and untranslated), and a feature file (GBK or GFF format).

### How to run?

The code below shows an example on how to run `prodigal`. The options used are:

| option/input | description |
| :-: | :- |
| `-i` | FASTA file to infer genes from (i.e. assembly FASTA file). |
| `-a` | Predicted genes (amino acid sequences). |
| `-d` | Predicted genes (nucleotide sequences). |
| `-o` | Output feature file. |
| `-f` | Format of output feature file. |

Running `prodigal -h` will show a description of other options available. You can also find the descriptions here: [Prodigal options cheat sheet](https://github.com/hyattpd/Prodigal/wiki/cheat-sheet)

`prodigal` is also available in the [EU Galaxy webserver](https://usegalaxy.eu/).

In [ ]:
!prodigal \
    -i assembly_contigs.fasta \
    -a prodigal_proteins.fasta \
    -d prodigal_nuc.fasta \
    -o prodigal.gff \
    -f gff

### What are its outputs?

As mentioned above, `prodigal` gives you FASTA files containing the predicted genes, both in untranslated (nucleotide) and translated format (amino acid). The command above will also output a feature file in GFF.

You will also see in these output files a string of fields or descriptors (e.g. `ID`, `partial`, `conf`, etc). The list below describes some of these fields:

1. `ID` - A unique ID assigned to a predicted gene.
2. `partial` - 00 = has a start and stop codon, 11 = no start and stop codon, 10 = no start but has stop codon, 01 = has a start but no stop codon.
3. `conf` - Confidence score of prediction
4. `start_type` - start codon sequence (Edge means no start codon).
5. `stop_type` - stop codon sequence (Edge means no stop codon).

You can find more detailed descriptions of these fields here: [Understanding the Prodigal output](https://github.com/hyattpd/Prodigal/wiki/understanding-the-prodigal-output)

You could also definitely use these fields to process your output files further. For instance, if you are just concerned with full genes (i.e. with start and stop codon), you can use the `awk` code below to select on the sequences that contain `partial=00`.

In [ ]:
!awk '/^>/ && /partial=00/ {flag=1} /^>/ && !/partial=00/ {flag=0} flag' prodigal_proteins.fasta > prodigal_proteins_full_gene.fasta

---
# <font color = 'gray'>AUGUSTUS</font>

Prokaryotes have simpler gene structure compared to eukaryotes because they do not contain introns. However, eukaryotes do, and so when predicting protein-coding genes in this domain, we cannot simply use softwares, such as `prodigal`, that are specifically catered for prokaryotic organisms. Furthermore, even within the eukaryotic lineage, different taxa can exhibit different patterns for gene splicing.

`augustus` is a utility that is widely used in predicting in eukaryotic genomes. Additionally, it provides gene models for various organisms which you could use to make your predictions more accurate.

### How to run?

An example command is displayed below to run `augustus` locally. Options used are:

| option/input | description |
| :-: | :- |
| `--species=` | Gene model pre-trained on the indicated species (`Ostreococcus_tauri` in the example below). You can find the list of the pre-trained models inside the `Augustus/config/species/` directory. |
| `--codingseq=on` | Include the coding sequence in the GTF output. |
| `--outfile=` | File to redirect standard output. |

More details about command parameters are described here: [Running Augustus](https://github.com/Gaius-Augustus/Augustus/blob/master/docs/RUNNING-AUGUSTUS.md)

They also have free-to-use webserver which you can find here: [AUGUSTUS webserver](https://bioinf.uni-greifswald.de/webaugustus/).

It is also available in the [EU Galaxy webserver](https://usegalaxy.eu/).

In [ ]:
!augustus \
    --species=Ostreococcus_tauri \
    --codingseq=on \
    --outfile=augusts_gene_predictions.gtf \
    assembly.fasta

### What are its outputs?

The only output of `augustus` is a **feature file (GTF)**. If you want to extract the predicted protein sequences, run the `awk` code below. Replace `protein sequence` by `coding sequence` in the third condition of the code if you want to extract the nucleotide coding sequences. Don't forget to replace filename of the redirected standard output (`augustus_proteins.fasta`).

In [6]:
%%bash
awk '\
    /start gene/ \
        {print ">"$4} \
    flag==1 && !/\]$/ \
        {print $2} \
    /protein sequence/ \
        {print substr($5,2,length($5)); flag=1} \
    /\]$/ && flag==1 \
        {print substr($2,1,length($2)-1); flag=0}' \
augustus_trial > augustus_proteins.fasta 

---
# <font color = 'gray'>Prokka</font>

Whole genome annotation is the process of identifying features of interest in a set of genomic DNA sequences, and labelling them with useful information. Prokka is a software tool to annotate **bacterial, archaeal and viral genomes** quickly and produce standards-compliant output files. (Source: [Prokka GitHub](https://github.com/tseemann/prokka))

### How to run?

`prokka` only requires one input file - the assembly file (`assembly.fasta`). By default, its annotation mode is set for **Bacteria**. The command below uses the following options:

| option/input | description |
| :-: | :- |
| `--outdir` | Output folder. |
| `--prefix` | Prefix of output files. |

If you want to explore more options, run `prokka -h`. Moreover, you could explore a wide array of `prokka` usage in their [GitHub page](https://github.com/tseemann/prokka).

In [ ]:
!prokka \
    --outdir prokka_out \
    --prefix prokka_func_annot \
    assembly.fasta

### What are its outputs?

Unlike `prodigal` which only provides sequences of predicted genes, `prokka` takes this further and performs homology search against their built-in databases as well. As such, in its outputs you can see not only the gene IDs but also the functional annotation hit.

You can find the description of the output files here: [Prokka output files](https://github.com/tseemann/prokka?tab=readme-ov-file#output-files).

The format of the output FASTA headers are also described here: [Prokka FASTA header format](https://github.com/tseemann/prokka?tab=readme-ov-file#fasta-database-format)

---
# <font color = 'gray'>BLAST</font>

BLAST is one of the most common bioinformatics tool used to perform local alignment of a query sequence against a set of reference sequences. When using this tool, another aspect equally important to consider is the database of reference sequences to which you will be comparing your query sequences against.

### Common Databases

There are a variety of reference database at your disposal. Some are vast and covers wide range of taxonomic and functional groups and could be used for more general applications. On the other hand, other databases are more targeted for specific domains. 

As with many other resources, not just in this context, there is generally a tradeoff: Databases with higher sensitivity (often due to their larger size) tend to have lower accuracy (often due to lower levels of curation), and vice versa. As a researcher, you must decide what matters most for your study. For instance, if you are specifically interested in plastic-degrading genes, it may be more relevant to use a dedicated database, such as PlasticDB (see below), instead of a database with wider scope.

Below are some common databases for functional annotation.

**NCBI nt database**

Non-redundant nucleotide and protein database. 
- Pros:
    1. Vastness of database will likely yield more hits. 
- Cons:
    1. Difficult to run on a local device due to computer resource limitations (i.e. Large database size, likely longer runtimes, etc).
    2. Level of curation of database.
    
**Swiss-Prot**

Protein database manually curated by experts. 

- Pros:
    1. Reliability due to level of curation
    2. Cross-references (Gene Ontology, MetaCyc, KEGG, Pfam, etc) makes it easier to analyze at different levels of function (i.e. gene level vs pathway level). 
- Cons:
    1. Lower number of reference sequences will likely result to lower annotation rates
        
**RefSeq**

NCBI’s Reference Sequence (RefSeq) database is a collection of taxonomically diverse, non-redundant and richly annotated sequences representing naturally occurring molecules of DNA, RNA, and protein. Included are sequences from plasmids, organelles, viruses, archaea, bacteria, and eukaryotes. (source: https://www.ncbi.nlm.nih.gov/books/NBK21091/). You can find taxonomically categorized RefSeq entries [here](https://ftp.ncbi.nlm.nih.gov/genomes/refseq/).

- Pros: 
    1. Has separate categories for different taxonomic lineages which allows a more targeted annotation.
- Cons:
    1. Can be biased on model organisms (which is also probably true, to some extent, in most databases).
    
**Custom database**

You can also create your own database. For instance, you can pull out annotated genomes and/or transcriptomes of organisms closely related to your target species from [JGI](https://genome.jgi.doe.gov/portal/) and use this as your reference database. If you are searching for a specific gene in your dataset, you can also pool together sequences of that gene from various databases or studies and use that as your custom reference database.

- Pros:
    1. More targeted.
- Cons:
    1. May need more pre and/or post-processing steps depending on the source of reference sequences.
        
**Other databases**

There are plenty of other databases dedicated for specific domains. A few of these are listed below.

- PlasticDB - Database of plastic-degrading enzymes (Link: [PlasticDB](https://plasticdb.org/)).
- Prokaryotic Antiviral Defense System - Database of prokaryotic antiviral defense systems related genes (Link: [PADS](https://ngdc.cncb.ac.cn/padsarsenal/main.php)).
- Carbohydrate Active Enzymes - Database containing families of structurally-related catalytic and carbohydrate-binding modules (or functional domains) of enzymes that degrade, modify, or create glycosidic bonds (Link: [CAZy](http://www.cazy.org/)).
- Toxic Exposome Database - Combines detailed toxin data with comprehensive toxin target information (Link: [T3DB](http://www.t3db.ca/)).
- Comprehensive Antibiotic Resistance Database - Database of resistance genes, their products and associated phenotypes (Link: [CARD](https://card.mcmaster.ca/)).


### How to run?

To run BLAST locally, first you need to have your reference nucleotide sequences. In here, we will assume that the file is in FASTA format and is named `reference_sequences.fasta`. We then have to create a BLAST database using the `makeblastdb` utility. A basic usage is shown below. If your reference sequences are composed of amino acid sequences, change `-dbtype nucl` to `-dbtype prot`.

This will output the indexed BLAST database inside the folder `BLAST_db/`.

Other platforms to run BLAST:
1. [NCBI BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi)
2. [EU Galaxy Webserver](https://usegalaxy.eu/)

In [ ]:
!makeblastdb \
    -in reference_sequences.fasta \
    -dbtype nucl \
    -out BLAST_db/reference_sequences

Afterwards, we can align our query sequences (`query_seqs.fasta`) against the sequences in the BLAST database as follows. We assume here that `query_seqs.fasta` was derived from the prediction of coding DNA sequences from the an assembly (e.g. using tools such as `prodigal` or `augustus`).

Arguments used:

| option/input | description |
| :-: | :- |
| `-query` |File containing query sequences. |
| `-db` | Prefix of BLAST database. |
| `-outfmt` | Output format. |
| `-out` | Output filename |
| `-evalue` | e-value threshold filter. |

In [ ]:
!blastn \
    -query query_seqs.fasta \
    -db BLAST_db/reference_sequences \
    -outfmt 6 \
    -out query_seqs_blast.out \
    -evalue 0.00001

### What are its outputs?

The output of the running BLAST locally depends on the format specified in `-outfmt`. You can see a short description of various BLAST output formats (as well as other arguments) here: [BLAST CLI](https://www.ncbi.nlm.nih.gov/books/NBK279684/table/appendices.T.options_common_to_all_blast/).

---
# <font color = 'gray'>DIAMOND</font>

DIAMOND is a sequence aligner for protein and translated DNA searches, designed for high performance analysis of big sequence data. The key features are:

- Pairwise alignment of proteins and translated DNA at **100x-10,000x speed of BLAST**.
- Protein clustering of up to tens of billions of proteins.
- Frameshift alignments for long read analysis.
- Low resource requirements and suitable for running on standard desktops or laptops.
- Various output formats, including BLAST pairwise, tabular and XML, as well as taxonomic classification.

Source: [DIAMOND GitHub page](https://github.com/bbuchfink/diamond)

### How to run?

Just like BLAST, we need a reference database of amino acid sequences to run DIAMOND. You can refer to the Databases part of the BLAST section above.

To create an index of the reference database, first we have to use the `diamond makedb` utility. We assume here that `reference_prot_sequences.fasta` is a FASTA file containing amino acid reference sequences.

In [ ]:
!diamond makedb \
    --in reference_prot_sequences.fasta \
    --db DIAMOND_db/reference_prot_sequences

Then, we can align the query sequences against the database as demonstrated below. Similar to BLAST, in the context of genomics, we assume that the query sequnces (`query_prot_seqs.fasta`) was inferred from the genome assembly using gene prediction tools.

Arguments used are:

| option/input | description |
| :-: | :- |
| `-d` | DIAMOND reference database generated by `diamond makedb`. |
| `-q` | FASTA file containing query protein sequences. |
| `--outfmt` | Output format. |
| `-out` | Output file. |
| `-k` | Only report the top _k_ hits. |
| `-e` | e-value threshold filter. |

Other parameters are described here: [DIAMOND CLI Options](https://github.com/bbuchfink/diamond/wiki/3.-Command-line-options).

In [ ]:
!diamond blastp \
    -d DIAMOND_db/reference_prot_sequences.dmnd \
    -q query_prot_seqs.fasta \
    --outfmt 6 \
    -o query_prot_seqs_diamond.out \
    -k 100 \
    -e 1e-5

### What are its outputs?

DIAMOND produces similar output to BLAST. See the section about BLAST above.

---
# <font color = 'gray'>EggNOG Mapper</font>

EggNOG-mapper is a tool for fast functional annotation of novel sequences. It uses precomputed orthologous groups and phylogenies from the eggNOG database to transfer functional information from fine-grained orthologs only.

The use of orthology predictions for functional annotation permits a higher precision than traditional homology searches (i.e. BLAST searches), as it avoids transferring annotations from close paralogs (duplicate genes with a higher chance of being involved in functional divergence).

Source: [EggNOG Mapper GitHub page](https://github.com/eggnogdb/eggnog-mapper)

### How to run?

EggNOG mapper can be accessed through an EMBL-hosted webserver located here: [EggNOG Mapper Webserver](https://eggnog-mapper.embl.de/). You simply need to upload your query sequences.

Alternatively, EggNOG mapper is also made available through the [EU Galaxy webserver](https://usegalaxy.eu/).

### What are its outputs?

The output is a tab-separated file containing the following columns.

1. `query_name`-  query sequence name
2. `seed_eggNOG_ortholog` - best protein match in eggNOG
3. `seed_ortholog_evalue` - best protein match (e-value)
4. `seed_ortholog_score` - best protein match (bit-score)
5. `predicted_taxonomic_group`
6. `predicted_protein_name` - Predicted protein name for query sequences
7. `GO_terms` - Comma delimited list of predicted Gene Ontology terms
8. `EC_number`
9. `KEGG_KO`
10. `KEGG_Pathway` - Comma delimited list of predicted KEGG pathways
11. `KEGG_Module`
12. `KEGG_Reaction`
13. `KEGG_rclass`
14. `BRITE`
15. `KEGG_TC`
16. `CAZy`
17. `BiGG_Reactions`
18. `Annotation_tax_scope` - The taxonomic scope used to annotate this query sequence
19. `Matching_OGs` - Comma delimited list of matching eggNOG Orthologous Groups
20. `best_OG|evalue|score` - Best matching Orthologous Groups (deprecated, use smallest from eggnog OGs)
21. `COG_functional_categories` - COG functional category inferred from best matching OG
22. `eggNOG_free_text_description`

As you can see, eggNOG mapper's output provides links to other resources which attempts to systematically group together biologically related functions (e.g. GO, KEGG, COG, etc.). This enables us to easily aggregate gene-level functions to higher-order categories (e.g. pathways) if needed. For instance, if you are interested in studying beta-lactam resistance, you can check if you have sequences with `KEGG_Pathway` annotation of ko01501 (Name: beta-Lactam resistance).

---
# <font color = 'gray'>BlastKOALA</font>

BlastKOALA (and GhostKOALA) are automatic annotation servers for genome and metagenome sequences, which perform KO (KEGG Orthology) assignments to characterize individual gene functions and reconstruct KEGG pathways, BRITE hierarchies and KEGG modules to infer high-level functions of the organism or the ecosystem.

Source: [BlastKOALA and GhostKOALA: KEGG Tools for Functional Characterization of Genome and Metagenome Sequences](https://www.sciencedirect.com/science/article/pii/S002228361500649X#s0010)

### How to run?

BlastKOALA is accessible through this link: [BlastKOALA webserver](https://www.kegg.jp/blastkoala/). Your input data must be a FASTA file of amino acid sequences (maximum of 50,000 query sequences). This can be obtained from gene-predicting tools discussed above such as `prodigal` and `augustus`. Then, you have to select the domain (i.e. eukaryote, prokaryote, virus) of your organisms of interest. You will also need an email address to confirm submission of your job.

### What are its outputs?

Briefly, the webserver produces a summary of annotated genes (i.e. annotation rate, pie chart). A table showing KO assignments of annotated genes is also provided. You could also reconstruct KEGG pathways based on the KO annotations.

A more concrete example of BlastKOALA's output is described here: [BlastKOALA step-by-step instructions](https://www.kegg.jp/blastkoala/help_blastkoala.html).

---
# <font color = 'gray'>MetaEuk</font>

Functional annotation of **shotgun metagenomic contigs** is straightforward for prokaryotic organisms due to their simple gene structure. Tools mentioned above like `prokka` and `prodigal` can be directly applied to this type of data. However, in eukaryotic metagenomic contigs, tools that are able to handle the more complex gene structures should be utilized when functionally annotating these sequences.

`metaeuk` is a bioinformatic tool dedicated for functional annotation of eukaryotic metagenomic contigs. Briefly, it searches for putative protein-coding genes by considering all six-frame translations. Subsequent cleaning and scoring steps are then applied to ensure that the inferred protein-coding genes are more likely to be correct.

### How to run?

To run `metaeuk`, we can use `metaeuk easy-predict` which wraps several steps of the `metaeuk` workflow into a single command. You need the following as inputs to the command:

| option/input | description |
| :-: | :- |
| `assembly.fasta` | Contig assembly file. |
| `ref_seqs.fasta` | Target sequences/reference sequences. |

For `ref_seqs.fasta`, you can choose a database that you think best suits your objectives. One great option that is relevant for MOLab are protein sequences predicted from **marine metagenomic datasets (Tara Oceans)** using `metaeuk`. You can find the reference sequences here: [MetaEuk Reference Sequences](https://wwwuser.gwdguser.de/~compbiol/metaeuk/).

In [ ]:
!metaeuk easy-predict \
    assembly.fasta \
    ref_seqs.fasta \
    metaeuk_out \
    tempFolder

### What are its outputs?

The outputs are:

1. FASTA file of predicted proteins.
2. Feature file (GFF).